In [15]:
import os
from io import StringIO
from google.colab import files
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import requests, gzip, shutil, unicodedata

In [4]:
# Upload do arquivo
uploaded = files.upload()

# Obter o nome do arquivo enviado
nome_arquivo = list(uploaded.keys())[0]


Saving Agendamentos_Tratados_Python.csv to Agendamentos_Tratados_Python.csv


In [5]:
# URL do dataset
url = "https://data.brasil.io/dataset/genero-nomes/nomes.csv.gz"
arquivo_compactado = "nomes.csv.gz"
arquivo_csv = "nomes.csv"

# Baixar o arquivo compactado
response = requests.get(url)
with open(arquivo_compactado, 'wb') as f:
    f.write(response.content)

# Descompactar o arquivo
with gzip.open(arquivo_compactado, 'rb') as f_in:
    with open(arquivo_csv, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Carregar o dataset em um DataFrame
df_nomes = pd.read_csv(arquivo_csv)


In [ ]:
#print(df.columns.tolist())

In [6]:
# Leitura do arquivo
df = pd.read_csv(nome_arquivo, sep=",")

# Garantir conversões corretas
df["Data"] = pd.to_datetime(df["Data"], dayfirst=True)
df["Cadastramento"] = pd.to_datetime(df["Cadastramento"], dayfirst=True)
df["Valor"] = pd.to_numeric(df["Valor"], errors="coerce")


# Extrair duração numérica em minutos
df["Duracao_min"] = df["Duração"].str.extract(r"(\d+)").astype(float)

# Normalização do nome
def normalizar_nome(nome):
    nome = unicodedata.normalize('NFKD', nome).encode('ASCII', 'ignore').decode('ASCII')
    return nome.strip().upper()

# Extrair primeiro nome do cliente e normalizar
df["PrimeiroNome"] = df["Cliente"].str.split().str[0]
df["Nome_Normalizado"] = df["PrimeiroNome"].apply(normalizar_nome)

# Preparar dataframe de nomes
nomes_mapeados = df_nomes[["first_name", "classification"]].dropna().drop_duplicates()
nomes_mapeados["first_name"] = nomes_mapeados["first_name"].apply(str.upper)

# Mesclar os dados com o gênero estimado
df = df.merge(nomes_mapeados, how="left", left_on="Nome_Normalizado", right_on="first_name")
df["Sexo"] = df["classification"].map({"M": "masculino", "F": "feminino"}).fillna("desconhecido")

In [ ]:
# @title Gráfico - Evolução de Agendamentos por Semana {"vertical-output":true,"display-mode":"form"}
# Agrupar por semana e extrair o início da semana como data
agendamentos_semanais = df.groupby(df["Data"].dt.to_period("W")).size().reset_index(name="Qtd")
agendamentos_semanais["Data"] = agendamentos_semanais["Data"].apply(lambda x: x.start_time)
agendamentos_semanais["Semana"] = agendamentos_semanais["Data"].dt.strftime("S%W/%y")

fig = px.line(
    agendamentos_semanais,
    x="Data", y="Qtd",
    title="Evolução de Agendamentos por Semana",
    labels={"Data": "Semana", "Qtd": "Agendamentos"},
    template="plotly_dark"
)

fig.update_traces(
    mode="lines+markers+text",
    texttemplate="%{customdata[0]}<br>Q: %{y}",
    textposition="top center",
    textfont_size=10,
    customdata=agendamentos_semanais[["Semana"]]
)

fig.update_yaxes(range=[0, agendamentos_semanais["Qtd"].max() * 1.2])
fig.show()


In [ ]:
# Valido - Mostrar total de cada servico
# @title Gráfico - Faturamento por Profissional {"vertical-output":true,"display-mode":"form"}

df_fat = df.groupby("Profissional")["Valor"].sum().reset_index()
df_fat["Valor_Label"] = "R$: " + df_fat["Valor"].round(2).astype(str)

fig = px.bar(
    df_fat,
    x="Profissional", y="Valor",
    title="Faturamento por Profissional",
    template="plotly_dark",
    text="Valor_Label"
)

fig.update_traces(
    textposition="outside",
    textfont_size=11,
    texttemplate="%{x}<br>%{text}"
)

# Expandir o eixo Y para dar espaço
max_val = df_fat["Valor"].max()
fig.update_yaxes(range=[0, max_val * 1.3])

# Aumentar altura total do gráfico
fig.update_layout(bargap=0.3, height=500)

fig.show()


In [ ]:
# Valido - Mostrar total de cada servico
# @title Gráfico - Faturamento por Profissional (Percentual) {"vertical-output":true,"display-mode":"form"}

df_fat = df.groupby("Profissional")["Valor"].sum().reset_index()
df_fat["Valor_Label"] = "R$: " + df_fat["Valor"].round(2).astype(str)


df_fat["Percentual"] = (df_fat["Valor"] / df_fat["Valor"].sum()) * 100
df_fat["Percentual_Label"] = df_fat["Percentual"].round(2).astype(str) + "%"

fig = px.bar(
    df_fat,
    x="Profissional",
    y="Percentual",
    title="Faturamento por Profissional (%)",
    template="plotly_dark",
    text="Percentual_Label"
)

fig.update_traces(
    textposition="outside",
    textfont_size=11,
    texttemplate="%{x}<br>%{text}"
)

fig.update_yaxes(range=[0, df_fat["Percentual"].max() * 1.3])
fig.update_layout(bargap=0.3, height=500)


In [ ]:
# @title Gráfico - Atendimentos por Profissional {"vertical-output":true,"display-mode":"form"}

df_qtd = df.groupby("Profissional").size().reset_index(name="Atendimentos")
df_qtd["Texto"] = df_qtd["Atendimentos"].astype(str)

fig = px.bar(
    df_qtd,
    x="Profissional",
    y="Atendimentos",
    title="Atendimentos por Profissional",
    template="plotly_dark",
    text="Texto"
)

fig.update_traces(
    textposition="outside",
    textfont_size=11,
    texttemplate="%{text}"
)

# Garantir espaço suficiente para os rótulos
fig.update_yaxes(range=[0, df_qtd["Atendimentos"].max() * 1.2])
fig.update_layout(bargap=0.3)

fig.show()


In [ ]:
# Juntar Corte Jeff para apenas Corte
# @title Gráfico - Serviços Mais Realizados {"vertical-output":true,"display-mode":"form"}

# Unificar categorias de serviços
def categorizar_servico(servico):
    servico = servico.lower()
    if "barba" in servico and "corte" in servico:
        return "Cabelo e Barba"
    elif "barba" in servico:
        return "Barba"
    elif "corte" in servico or "juliano" in servico or "promocional" in servico:
        return "Cabelo"
    else:
        return "Outros"

df["Serviço Simplificado"] = df["Serviço"].apply(categorizar_servico)

df_serv = df.groupby("Serviço Simplificado").size().reset_index(name="Quantidade")

fig = px.bar(df_serv.sort_values("Quantidade", ascending=False),
             x="Serviço Simplificado", y="Quantidade",
             title="Serviços Mais Realizados (Categorias Unificadas)",
             template="plotly_dark")
fig.update_layout(bargap=0.3)
fig.show()


In [ ]:
# Colocar valor total e labels
# @title Gráfico - Faturamento por Semana (por Profissional) {"vertical-output":true,"display-mode":"form"}

faturamento_semana_prof = df.groupby([df["Data"].dt.to_period("W"), "Profissional"])["Valor"].sum().reset_index()
faturamento_semana_prof["Data"] = faturamento_semana_prof["Data"].apply(lambda p: p.start_time)
faturamento_semana_prof["Semana"] = faturamento_semana_prof["Data"].dt.strftime("S%W/%y")

faturamento_semana_prof = faturamento_semana_prof[faturamento_semana_prof["Semana"] != "S16/25"]

fig = px.bar(
    faturamento_semana_prof,
    x="Semana", y="Valor",
    color="Profissional",
    title="Faturamento por Semana (por Profissional)",
    labels={"Semana": "Semana", "Valor": "Faturamento (R$)"},
    template="plotly_dark"
)

fig.update_layout(barmode="stack")
fig.show()


In [ ]:
# Colocar valor total e labels
# @title Gráfico - Faturamento por Semana % (por Profissional) {"vertical-output":true,"display-mode":"form"}

faturamento_semana_prof = df.groupby([df["Data"].dt.to_period("W"), "Profissional"])["Valor"].sum().reset_index()
faturamento_semana_prof["Data"] = faturamento_semana_prof["Data"].apply(lambda p: p.start_time)
faturamento_semana_prof["Semana"] = faturamento_semana_prof["Data"].dt.strftime("S%W/%y")

faturamento_semana_prof = faturamento_semana_prof[faturamento_semana_prof["Semana"] != "S16/25"]


total_semana = faturamento_semana_prof.groupby("Semana")["Valor"].transform("sum")
faturamento_semana_prof["Percentual"] = (faturamento_semana_prof["Valor"] / total_semana) * 100

fig = px.bar(
    faturamento_semana_prof,
    x="Semana",
    y="Percentual",
    color="Profissional",
    title="Faturamento por Semana (por Profissional) - Percentual",
    labels={"Semana": "Semana", "Percentual": "Faturamento (%)"},
    template="plotly_dark"
)

fig.update_layout(barmode="stack", height=500)
fig.show()


In [10]:
# @title Gráfico - Horas Trabalhadas por Profissional {"vertical-output":true,"display-mode":"form"}

# Calcular horas a partir de minutos
horas_trabalhadas = df.groupby("Profissional")["Duracao_min"].sum().reset_index()
horas_trabalhadas["Horas"] = (horas_trabalhadas["Duracao_min"] / 60).round(0)
horas_trabalhadas["Horas_str"] = horas_trabalhadas["Horas"].astype(int).astype(str) + "h"

# Ordenar para o gráfico
horas_trabalhadas_sorted = horas_trabalhadas.sort_values("Horas", ascending=False)

# Criar gráfico
fig = px.bar(
    horas_trabalhadas_sorted,
    x="Profissional",
    y="Horas",
    title="Horas Trabalhadas por Profissional",
    labels={"Horas": "Horas Trabalhadas"},
    text=horas_trabalhadas_sorted["Horas_str"],
    template="plotly_dark"
)

fig.update_layout(bargap=0.3)
fig.show()



In [11]:
# @title Gráfico - Agendamentos por Dia da Semana e Mês {"vertical-output":true,"display-mode":"form"}


# Criar colunas auxiliares
df["mes"] = df["Data"].dt.strftime("%m/%y")
df["dia_da_semana_en"] = df["Data"].dt.day_name()

traducao_dias = {
    'Monday': 'segunda-feira',
    'Tuesday': 'terça-feira',
    'Wednesday': 'quarta-feira',
    'Thursday': 'quinta-feira',
    'Friday': 'sexta-feira',
    'Saturday': 'sábado',
    'Sunday': 'domingo'
}
df["dia_da_semana"] = df["dia_da_semana_en"].map(traducao_dias)

# Contar agendamentos
heatmap_data = df.groupby(['mes', 'dia_da_semana']).size().reset_index(name='agendamentos')
heatmap_pivot = heatmap_data.pivot(index='dia_da_semana', columns='mes', values='agendamentos').fillna(0)

# Ordenar os dias da semana
dias_ordem = ['segunda-feira', 'terça-feira', 'quarta-feira', 'quinta-feira', 'sexta-feira', 'sábado', 'domingo']
heatmap_pivot = heatmap_pivot.reindex(dias_ordem)

# Criar gráfico
fig_heatmap = px.imshow(
    heatmap_pivot,
    labels=dict(x='Mês', y='Dia da Semana', color='Qtd. Agendamentos'),
    x=heatmap_pivot.columns,
    y=heatmap_pivot.index,
    color_continuous_scale='YlOrRd',
    text_auto='.0f',
    aspect='auto',
    title='Agendamentos por Dia da Semana e Mês'
)

# Aplicar fundo escuro
fig_heatmap.update_layout(
    margin=dict(l=40, r=40, t=50, b=40),
    plot_bgcolor='#111111',
    paper_bgcolor='#111111',
    font_color='white'
)

fig_heatmap.show()


In [12]:
# @title Gráfico - Faturamento por Profissional por Semana (sem a última semana) {"vertical-output":true,"display-mode":"form"}


# Garantir o tipo datetime correto
df["Data"] = pd.to_datetime(df["Data"], format="%d/%m/%Y")

# Extrair ano e semana ISO
df["Ano"] = df["Data"].dt.isocalendar().year
df["Semana"] = df["Data"].dt.isocalendar().week
df["Ano_Semana"] = df["Ano"].astype(str) + "-S" + df["Semana"].astype(str).str.zfill(2)

# Remover a última semana do ano mais recente
ultima_semana = df["Semana"].max()
ultimo_ano = df["Ano"].max()
df_filtrado = df[~((df["Ano"] == ultimo_ano) & (df["Semana"] == ultima_semana))]

# Agrupar por semana e profissional (somando o faturamento)
faturamento_semana = df_filtrado.groupby(["Ano_Semana", "Profissional"])["Valor"].sum().reset_index()
faturamento_semana["Texto"] = faturamento_semana["Valor"].round(0).astype(int).astype(str)

# Criar gráfico
fig = px.bar(
    faturamento_semana,
    x="Ano_Semana",
    y="Valor",
    color="Profissional",
    barmode="group",
    text="Texto",
    title="Faturamento por Profissional por Semana (sem a última semana)",
    labels={"Valor": "Faturamento (R$)", "Ano_Semana": "Semana"},
    template="plotly_dark"
)

# Ajustes visuais
fig.update_traces(
    textposition="outside",
    textfont_size=10,
    texttemplate="R$ %{text}"
)

fig.update_layout(
    bargap=0.2,
    xaxis_title="Semana (Ano-Semana)",
    yaxis_title="Faturamento (R$)"
)
fig.update_yaxes(range=[0, faturamento_semana["Valor"].max() * 1.2])

fig.show()


In [13]:
# @title Gráfico - Faturamento por Profissional por Semana % (sem a última semana) {"vertical-output":true,"display-mode":"form"}
# Garantir o tipo datetime correto
df["Data"] = pd.to_datetime(df["Data"], format="%d/%m/%Y")

# Extrair ano e semana ISO
df["Ano"] = df["Data"].dt.isocalendar().year
df["Semana"] = df["Data"].dt.isocalendar().week
df["Ano_Semana"] = df["Ano"].astype(str) + "-S" + df["Semana"].astype(str).str.zfill(2)

# Remover a última semana do ano mais recente
ultima_semana = df["Semana"].max()
ultimo_ano = df["Ano"].max()
df_filtrado = df[~((df["Ano"] == ultimo_ano) & (df["Semana"] == ultima_semana))]

# Agrupar por semana e profissional
faturamento_semana = df_filtrado.groupby(["Ano_Semana", "Profissional"])["Valor"].sum().reset_index()


total_por_semana = faturamento_semana.groupby("Ano_Semana")["Valor"].transform("sum")
faturamento_semana["Percentual"] = (faturamento_semana["Valor"] / total_por_semana) * 100
faturamento_semana["Texto"] = faturamento_semana["Percentual"].round(1).astype(str) + "%"

# Criar gráfico
fig = px.bar(
    faturamento_semana,
    x="Ano_Semana",
    y="Percentual",
    color="Profissional",
    barmode="group",
    text="Texto",
    title="Faturamento por Profissional por Semana (%)",
    labels={"Percentual": "Faturamento (%)", "Ano_Semana": "Semana"},
    template="plotly_dark"
)

# Ajustes visuais
fig.update_traces(
    textposition="outside",
    textfont_size=10,
    texttemplate="%{text}"
)

fig.update_layout(
    bargap=0.2,
    xaxis_title="Semana (Ano-Semana)",
    yaxis_title="Faturamento (%)"
)
fig.update_yaxes(range=[0, 100])  # Percentuais de 0% a 100%

fig.show()

In [14]:
# @title Gráfico - Atendimento por Profissional por Semana (sem a última semana) {"vertical-output":true,"display-mode":"form"}


df["Data"] = pd.to_datetime(df["Data"], format="%d/%m/%Y")

# Extrair ano e semana ISO
df["Ano"] = df["Data"].dt.isocalendar().year
df["Semana"] = df["Data"].dt.isocalendar().week
df["Ano_Semana"] = df["Ano"].astype(str) + "-S" + df["Semana"].astype(str).str.zfill(2)

# Remover a última semana
ultima_semana = df["Semana"].max()
ultimo_ano = df["Ano"].max()
df_filtrado = df[~((df["Ano"] == ultimo_ano) & (df["Semana"] == ultima_semana))]

# Agrupar novamente por semana e profissional
df_semana = df_filtrado.groupby(["Ano_Semana", "Profissional"]).size().reset_index(name="Atendimentos")
df_semana["Texto"] = df_semana["Atendimentos"].astype(str)

# Criar gráfico
fig = px.bar(
    df_semana,
    x="Ano_Semana",
    y="Atendimentos",
    color="Profissional",
    barmode="group",
    text="Texto",
    title="Atendimentos por Profissional por Semana (sem a última semana)",
    template="plotly_dark"
)

fig.update_traces(
    textposition="outside",
    textfont_size=10,
    texttemplate="%{text}"
)

fig.update_layout(
    bargap=0.2,
    xaxis_title="Semana (Ano-Semana)",
    yaxis_title="Atendimentos"
)
fig.update_yaxes(range=[0, df_semana["Atendimentos"].max() * 1.2])

fig.show()

In [ ]:
# @title Gráfico - Faturamento por Profissional por Mês (sem o último Mês) {"vertical-output":true,"display-mode":"form"}

# Garantir o tipo datetime correto
df["Data"] = pd.to_datetime(df["Data"], format="%d/%m/%Y")

# Extrair ano e mês
df["Ano"] = df["Data"].dt.year
df["Mes"] = df["Data"].dt.month
df["Ano_Mes_dt"] = df["Data"].dt.to_period("M").dt.to_timestamp()
df["Ano_Mes"] = df["Ano_Mes_dt"].dt.strftime("%b/%Y")  # Ex: Jan/2025

# Remover o último mês detectado
ultimo_ano = df["Ano"].max()
ultimo_mes = df[df["Ano"] == ultimo_ano]["Mes"].max()
df_filtrado = df[~((df["Ano"] == ultimo_ano) & (df["Mes"] == ultimo_mes))]

# Agrupar faturamento por mês e profissional
faturamento_mes = df_filtrado.groupby(["Ano_Mes", "Profissional"])["Valor"].sum().reset_index()
faturamento_mes["Texto"] = faturamento_mes["Valor"].round(0).astype(int).astype(str)

# Criar gráfico
fig = px.bar(
    faturamento_mes,
    x="Ano_Mes",
    y="Valor",
    color="Profissional",
    barmode="group",
    text="Texto",
    title="Faturamento por Profissional por Mês (sem o último mês)",
    labels={"Valor": "Faturamento (R$)", "Ano_Mes": "Mês"},
    template="plotly_dark",
    category_orders={"Ano_Mes": sorted(faturamento_mes["Ano_Mes"].unique(), key=lambda x: pd.to_datetime(x, format="%b/%Y"))}
)

# Ajustar texto e layout
fig.update_traces(
    textposition="outside",
    textfont_size=10,
    texttemplate="R$ %{text}"
)

fig.update_layout(
    bargap=0.2,
    xaxis_title="Mês (Abrev./Ano)",
    yaxis_title="Faturamento (R$)"
)
fig.update_yaxes(range=[0, faturamento_mes["Valor"].max() * 1.2])

fig.show()


In [ ]:
# @title Gráfico - Faturamento por Profissional por Mês % (sem o último Mês) {"vertical-output":true,"display-mode":"form"}

# Garantir o tipo datetime correto
df["Data"] = pd.to_datetime(df["Data"], format="%d/%m/%Y")

# Extrair ano e mês
df["Ano"] = df["Data"].dt.year
df["Mes"] = df["Data"].dt.month
df["Ano_Mes_dt"] = df["Data"].dt.to_period("M").dt.to_timestamp()
df["Ano_Mes"] = df["Ano_Mes_dt"].dt.strftime("%b/%Y")  # Ex: Jan/2025

# Remover o último mês detectado
ultimo_ano = df["Ano"].max()
ultimo_mes = df[df["Ano"] == ultimo_ano]["Mes"].max()
df_filtrado = df[~((df["Ano"] == ultimo_ano) & (df["Mes"] == ultimo_mes))]

# Agrupar faturamento por mês e profissional
faturamento_mes = df_filtrado.groupby(["Ano_Mes", "Profissional"])["Valor"].sum().reset_index()

total_por_mes = faturamento_mes.groupby("Ano_Mes")["Valor"].transform("sum")
faturamento_mes["Percentual"] = (faturamento_mes["Valor"] / total_por_mes) * 100
faturamento_mes["Texto"] = faturamento_mes["Percentual"].round(1).astype(str) + "%"

fig = px.bar(
    faturamento_mes,
    x="Ano_Mes",
    y="Percentual",
    color="Profissional",
    barmode="group",
    text="Texto",
    title="Faturamento por Profissional por Mês (%)",
    labels={"Percentual": "Faturamento (%)", "Ano_Mes": "Mês"},
    template="plotly_dark",
    category_orders={"Ano_Mes": sorted(faturamento_mes["Ano_Mes"].unique(), key=lambda x: pd.to_datetime(x, format="%b/%Y"))}
)


fig.update_traces(
    textposition="outside",
    textfont_size=10,
    texttemplate="%{text}"
)

fig.update_layout(
    bargap=0.2,
    xaxis_title="Mês (Abrev./Ano)",
    yaxis_title="Faturamento (%)"
)
fig.update_yaxes(range=[0, 100])

fig.show()
